# [과제 1] 성능 비교

In [1]:
import sys
import pickle
from pprint import pprint 
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from data.make_casting_graph import oneway_to_bidirected_graph
from scipy.sparse import csc_matrix
import time
#from pagerank import pagerank
from sklearn.preprocessing import normalize
from pyvis.network import Network

In [2]:
# create idx to num comments
with open('./data/ratings.csv', encoding='utf-8') as f:
    docs = [line.strip().split(',') for line in f.readlines()[1:]]
    _idx2numcomments = {movie_idx:int(num) for num, movie_idx in docs}

In [3]:
# pre defined casting weight graph
with open('./data/casting_graph.pkl', 'rb') as f:
    graph = pickle.load(f)

In [4]:
# create idx to actor name function
with open('./data/actors.csv', encoding='utf-8') as f:
    next(f)
    docs = [line.split(',') for line in f.readlines()[1:]]
    # English name if exist else Korean name
    _idx2actor = {doc[0]:doc[1] for doc in docs}

In [5]:
with open('./data/movies.csv', encoding='utf-8') as f:
    next(f)
    docs = [line.split(',') for line in f.readlines()[1:]]
    _idx2movie = {doc[0]:doc[1] for doc in docs if len(docs)}

In [6]:
idx2movie = lambda idx: _idx2movie.get(idx, 'Unknown')
idx2actor = lambda idx: _idx2actor.get(idx, 'Unknown')
idx2numcomments = lambda idx: _idx2numcomments.get(idx,0)

In [7]:
g = oneway_to_bidirected_graph(graph)

In [8]:
for movie in sorted(_idx2numcomments.items(), key=lambda x: x[1], reverse=True)[:10]:
    print(idx2movie(movie[0]),movie[1])

기생충 40
극한직업 15
마약왕 15
인터스텔라 14
어벤져스: 엔드게임 12
걸캅스 12
마녀 12
택시운전사 11
배심원들 11
신과함께-죄와 벌 11


In [9]:
def make_graph(casting_csv_path):
    # load file
    with open(casting_csv_path, encoding='utf-8') as f:
        next(f)
        graph = {line.split('\t')[0]:line.split('\t')[1].strip().split() for line in f if len(line.split('\t'))==2}
    
    # weighting
    # casting order (n-i)^2/ sum (i^2 for i = 1 to n)
    def weight(casting_order):
        if not casting_order:
            return {}
        n = len(casting_order)
        weights = [(n-i) ** 2 for i in range(n)]
        sum_ = sum(weights)
        return {actor:w/sum_ for actor, w in zip(casting_order, weights)}
    
    graph = {movie:weight(actors) for movie, actors in graph.items() if actors}
    return graph

In [10]:
def _normalize(G):
    """It returns outbound normalized graph
    Arguments
    ---------
    G: inbound graph dict of dict
    """
    # Sum of outbound weight
    # t: to node, f: from node, w: weight
    W_sum = {}    
    for t, f_dict in G.items():
        for f, w in f_dict.items():
            W_sum[f] = W_sum.get(f, 0) + w
    A = {t:{f:w/W_sum[f] for f,w in f_dict.items()} for t, f_dict in G.items()}    
    nodes = set(G.keys())
    nodes.update(W_sum)
    return A, nodes

In [11]:
def pagerank(G, bias=None, df=0.15,
             max_iter=50, converge_error=0.001,verbose=0):
    """
    Arguments
    ---------
    G: Inbound graph, dict of dict
        G[to_node][from_node] = weight (float)
    df: damping factor, float. default 0.15
    """
    
    A, nodes = _normalize(G)
    N = len(nodes) # number of nodes
    sr = 1 - df # survival rate (1 -  damping factor)
    ir = 1 / N # initial rank

    # Initialization
    rank = {n:ir for n in nodes}

    # Initialization of bias
    if not bias:
        bias = {node:ir for node in nodes}

    # Iteration
    for _iter in range(1, max_iter + 1):
        rank_new = {}

        # t: to node, f: from node, w: weight
        for t in nodes:
            f_dict = A.get(t, {})
            rank_t = sum((w*rank[f] for f, w in f_dict.items())) if f_dict else 0
            rank_t = sr * rank_t + df * bias.get(t, 0)
            rank_new[t] = rank_t

        # convergence check
        diff = sum((abs(rank[n] - rank_new[n]) for n in nodes))
        if diff < converge_error:
            if verbose:
                print('Early stopped at iter = {}'.format(_iter))
            break

        if verbose:
            sum_ = sum(rank_new.values())
            print('Iteration = {}, diff = {}, sum = {}'.format(_iter, diff, sum_))

        rank = rank_new

    return rank

In [12]:
import time
starttime = time.time()
bias = {node:(idx2numcomments(node.split()[1]) if node[0]== 'm' else 0) for node in g}
_sum = sum(bias.values())
bias = {node:b / _sum for node, b in bias.items()}
rank_d = pagerank(g, bias=bias,df=0.15,max_iter=30,converge_error=0.001,verbose=1)
print('computation time : {0}sec'.format(time.time()-starttime))

Iteration = 1, diff = 0.6745935594038663, sum = 1.0000000000000053
Iteration = 2, diff = 0.5133755765513067, sum = 1.0000000000000062
Iteration = 3, diff = 0.40708434710252966, sum = 1.000000000000008
Iteration = 4, diff = 0.328811456904488, sum = 1.000000000000002
Iteration = 5, diff = 0.26900006261697385, sum = 1.000000000000008
Iteration = 6, diff = 0.22172923044566595, sum = 0.999999999999992
Iteration = 7, diff = 0.18372765496993076, sum = 0.9999999999999929
Iteration = 8, diff = 0.15290648077655544, sum = 1.000000000000006
Iteration = 9, diff = 0.12756391624362162, sum = 0.9999999999999907
Iteration = 10, diff = 0.10676563571706416, sum = 0.9999999999999944
Iteration = 11, diff = 0.0894733554563146, sum = 1.0000000000000073
Iteration = 12, diff = 0.07517014319662879, sum = 1.000000000000009
Iteration = 13, diff = 0.063185288111448, sum = 0.9999999999999923
Iteration = 14, diff = 0.05320609097840663, sum = 0.999999999999991
Iteration = 15, diff = 0.0448304779270672, sum = 1.000000

In [13]:
import numpy as np
from scipy.sparse import csc_matrix

nodes = set(g.keys())
idx2node = list(sorted(nodes))
node2idx = {node:idx for idx, node in enumerate(idx2node)}

bias = np.asarray([b for node, b in sorted(bias.items(), key=lambda tp:node2idx[tp[0]])])
print(bias.shape)

rows = []
cols = []
data = []

for from_node, to_dict in g.items():
    from_idx = node2idx[from_node]
    for to_node, weight in to_dict.items():
        to_idx = node2idx[to_node]
        rows.append(from_idx)
        cols.append(to_idx)
        data.append(weight)
        
A = csc_matrix((data, (rows,cols)))
print(A.shape)

(6154,)
(6154, 6154)


In [14]:
import time
max_iter = 30
df = 0.85
starttime = time.time()

ir = 1 / A.shape[0]
rank = np.asarray([ir] * A.shape[0])

for n_iter in range(1, max_iter+1):
    rank_new = A.dot(rank)
    rank_new = normalize(rank_new.reshape(1,-1),norm='l1').reshape(-1)
    rank_new = df * rank_new + (1-df)*bias
    diff = abs(rank - rank_new).sum()
    rank = rank_new
    print('iter {} : diff = {}'.format(n_iter,diff))
print('computation time : {0}sec'.format(time.time()-starttime))

iter 1 : diff = 0.1685245368865779
iter 2 : diff = 0.123534416788289
iter 3 : diff = 0.11717242074154521
iter 4 : diff = 0.08676250638774644
iter 5 : diff = 0.08106650827175174
iter 6 : diff = 0.06044614044638538
iter 7 : diff = 0.05589952786903922
iter 8 : diff = 0.04188475454126574
iter 9 : diff = 0.038452782327255894
iter 10 : diff = 0.0289095171904886
iter 11 : diff = 0.026405522194198443
iter 12 : diff = 0.01994486388644759
iter 13 : diff = 0.01811137289916391
iter 14 : diff = 0.013753287448751986
iter 15 : diff = 0.012408911428306675
iter 16 : diff = 0.009469243738374537
iter 17 : diff = 0.008494000468005527
iter 18 : diff = 0.006511648928942716
iter 19 : diff = 0.005809774127703195
iter 20 : diff = 0.004473307017566352
iter 21 : diff = 0.0039712967053357525
iter 22 : diff = 0.0030704578506105173
iter 23 : diff = 0.0027152845982687866
iter 24 : diff = 0.002106149459828414
iter 25 : diff = 0.0018577039374234091
iter 26 : diff = 0.0014438021951808503
iter 27 : diff = 0.001270456142

# [과제 2] 영화 Top 10

In [15]:
rank_d

{'movie 153642': 0.00027221084258117497,
 'actor 2453': 0.0002826311330296402,
 'actor 3400': 3.090126295804653e-05,
 'movie 158180': 0.00045643056752668924,
 'movie 152266': 0.0001059268928105408,
 'movie 19051': 0.00010592689281054082,
 'movie 151674': 0.0002106138181861142,
 'actor 1490': 0.00019638794794307396,
 'actor 2487': 0.00016473279934373463,
 'actor 701': 0.00013261752467897726,
 'actor 1815': 0.0001772065184657108,
 'actor 2378': 0.00012054546708526984,
 'movie 38421': 0.00010226912083393767,
 'movie 17534': 0.0001059268928105408,
 'movie 142822': 0.0004187472036232306,
 'movie 53742': 0.00017787688060878307,
 'actor 3254': 5.250597677034909e-05,
 'actor 2112': 3.368000374578434e-05,
 'actor 732': 0.00014346033284843071,
 'actor 3073': 1.840874221865475e-05,
 'actor 3250': 0.00018428317049647188,
 'actor 1382': 5.6553305022723265e-05,
 'actor 349': 0.00010165157716281244,
 'movie 52524': 0.00021866006085167154,
 'actor 2226': 6.056255848112566e-05,
 'actor 432': 9.74187417

In [16]:
for key in list(rank_d.keys()):
    if 'actor' in key:
        del rank_d[key]
    
    

In [17]:
rank_d

{'movie 153642': 0.00027221084258117497,
 'movie 158180': 0.00045643056752668924,
 'movie 152266': 0.0001059268928105408,
 'movie 19051': 0.00010592689281054082,
 'movie 151674': 0.0002106138181861142,
 'movie 38421': 0.00010226912083393767,
 'movie 17534': 0.0001059268928105408,
 'movie 142822': 0.0004187472036232306,
 'movie 53742': 0.00017787688060878307,
 'movie 52524': 0.00021866006085167154,
 'movie 35624': 0.00016895247776673217,
 'movie 124225': 0.00010592689281054079,
 'movie 86888': 0.00017228685568666403,
 'movie 85857': 0.00011008178242213562,
 'movie 65540': 0.00016500500181195268,
 'movie 64995': 0.00013969652884822625,
 'movie 105015': 0.00024964883507741596,
 'movie 117281': 0.00029268572774349485,
 'movie 24196': 0.0002300950231391264,
 'movie 119294': 0.0004063298085710969,
 'movie 147092': 0.00010592689281054079,
 'movie 74303': 0.000144294414406207,
 'movie 52075': 0.0006168117998357099,
 'movie 156565': 9.525810667662356e-05,
 'movie 127382': 0.00023030641774765133

In [18]:
top_d = sorted(rank_d.items(), key=lambda t : t[1], reverse=True)[:10]
top_d

[('movie 161967', 0.0032033878121671224),
 ('movie 167651', 0.0014303471787626468),
 ('movie 175322', 0.0011565783119412997),
 ('movie 156464', 0.0011527961465662747),
 ('movie 130966', 0.001098819013448319),
 ('movie 177483', 0.0009469824923736168),
 ('movie 174065', 0.0009354687095915042),
 ('movie 37886', 0.000918249213245038),
 ('movie 154449', 0.00091821747845663),
 ('movie 163788', 0.0007997936563664337)]

In [19]:
top_user = pd.DataFrame(columns=("movieId", "title", "rank"))
for i in range(len(top_d)):
    top_user.loc[i] = {'rank': top_d[i][1],'movieId':top_d[i][0][6:],'title':''}


In [20]:
with open('./data/movies.csv', encoding='utf-8') as f:
    docs = [line.strip().split(',') for line in f.readlines()[1:]]

name =list()

for i in range(len(top_user)):
    for j in range(len(docs)):
        if top_user['movieId'][i] == docs[j][0]:
            name.append(docs[j][1])
name

['기생충',
 '극한직업',
 '마녀',
 '보헤미안 랩소디',
 '부산행',
 '배심원들',
 '걸캅스',
 '클레멘타인',
 '리틀 포레스트',
 '알라딘']

In [21]:
top_user['title'] = name
top_user

movieId     title      rank
0  161967       기생충  0.003203
1  167651      극한직업  0.001430
2  175322        마녀  0.001157
3  156464  보헤미안 랩소디  0.001153
4  130966       부산행  0.001099
5  177483      배심원들  0.000947
6  174065       걸캅스  0.000935
7   37886     클레멘타인  0.000918
8  154449   리틀 포레스트  0.000918
9  163788       알라딘  0.000800

In [22]:
top_user_n = pd.DataFrame(columns=("movieId", "title", "rank"))

In [29]:
movie_title=[]
movie_rank=[]

rank_ = {idx2node[idx]:value for idx, value in enumerate(rank)}
movierank = {node:value for node, value in rank_.items() if 'movie' in node}

for movie, value in sorted(movierank.items(), key=lambda x:-x[1])[:50]:
    movie_idx = movie.split()[1]
    movie_title.append(idx2movie(movie_idx))
    movie_rank.append(value)
    
top_user_n['title'] = movie_title
top_user_n['rank'] = movie_rank
top_user_n = top_user_n.head(10)
top_user_n

{'actor 1': 8.370457723797371e-05, 'actor 10': 0.00021801126276767329, 'actor 100': 8.370457723797368e-05, 'actor 1000': 8.37045772379737e-05, 'actor 1001': 0.00016635952195086926, 'actor 1002': 0.00019337278055119795, 'actor 1003': 0.00020001270919438473, 'actor 1004': 8.37045772379737e-05, 'actor 1005': 0.0002388290800096429, 'actor 1006': 0.0002120563829549753, 'actor 1007': 0.0003088390526906288, 'actor 1008': 0.00010780993707787115, 'actor 1009': 0.00015960722461256122, 'actor 101': 0.0001621228777568408, 'actor 1010': 8.370457723797371e-05, 'actor 1011': 0.00015656357773504435, 'actor 1012': 0.00015762460736727486, 'actor 1013': 8.82858348849561e-05, 'actor 1014': 0.00015041266527449199, 'actor 1015': 8.37045772379737e-05, 'actor 1016': 9.268779779252265e-05, 'actor 1017': 8.370457723797371e-05, 'actor 1018': 0.00014271117468079656, 'actor 1019': 0.00011175479785503471, 'actor 102': 0.00016311832159037642, 'actor 1020': 0.00030531895926456845, 'actor 1021': 9.007345094278106e-05,

ValueError: Length of values does not match length of index

In [24]:
movie_id=[]
for i in range(len(top_user_n)):
    for j in range(len(docs)):
        if top_user_n['title'][i] == docs[j][1]:
            movie_id.append(docs[j][0])
movie_id.pop()
top_user_n['movieId'] = movie_id
top_user_n

movieId       title      rank
0  161967         기생충  0.001544
1  156464    보헤미안 랩소디  0.001086
2  175322          마녀  0.000895
3  174065         걸캅스  0.000856
4  167651        극한직업  0.000765
5   37886       클레멘타인  0.000729
6  157297         마약왕  0.000713
7   71509         아저씨  0.000694
8  136900  어벤져스: 엔드게임  0.000657
9  163788         알라딘  0.000639

# [과제 3] 영화 Top 10 노드 시각화

In [25]:
act = pd.read_csv('./data/actors.csv')
act

id      name
0        1       NaN
1        2     혼고카나타
2        3    마크듀플라스
3        4   제이슨베이트먼
4        5    엘리마리엔탈
5        6  제이콥트렘블레이
6        7       김성오
7        8        호야
8        9      밋첼머소
9       10       김남주
10      11     루시보인턴
11      12       이경영
12      13       김민희
13      14       박시연
14      15       현태호
15      16      마드하반
16      17      사라로머
17      18    알렉산더니퐁
18      19      톰윌킨슨
19      20     이세마리야
20      21  크리스토퍼플러머
21      22       김향기
22      23      에이단퀸
23      24   구잘투르수노바
24      25       장신영
25      26       윤아민
26      27     비고모텐슨
27      28    프랭크란젤라
28      29       박은혜
29      30    조쉬할로웨이
...    ...       ...
3427  3428       민복기
3428  3429     게리루이스
3429  3430        조안
3430  3431     맷크레이븐
3431  3432       신현빈
3432  3433      닛타에미
3433  3434    캐서린모리스
3434  3435    나카무라치에
3435  3436    펠리시티존스
3436  3437   야마구치토모코
3437  3438       박보영
3438  3439   리처드판스워드
3439  3440   그레고리하인즈
3440  3441     매튜맥퍼딘
3441  3442       김부선
3442  3443       류경수
3443  3444      안톤옐친
3444  3445       심은경
3445  3446       하정우
3446  3447       배슬기
3447  3448    피오마르마이
3448  3449     난죠요시노
3449  3450       이태경
3450  3451       유재명
3451  3452   브리짓모이나한
3452  3453     리암에이켄
3453  3454       존시나
3454  3455       톰펠튼
3455  3456       박신양
3456  3457       김보연

[3457 rows x 2 columns]

In [26]:
from pyvis.network import Network
import pandas as pd

got_net = Network(height="750px", width="100%", bgcolor="#222222", font_color="white")
got_net.barnes_hut()

#Top10에 대한 노드를 추가
for row in top_user.iterrows() :
    src = row[1].title
    got_net.add_node(src, src)
    a = 'movie '+row[1].movieId
    #Actor를 노드 추가
    for i in g[a].keys():
        n = act[act['id'].isin([i[6:]])]['name'].values[0]
        got_net.add_node(n, n)
        got_net.add_edge(src, n)
        #Actor가 출연한 영화에 대한 노드를 만들어서 추가
        for j in g[i].keys():
            for k in range(len(docs)):
                if docs[k][0] == j[6:]:
                    got_net.add_node(docs[k][1], docs[k][1])
                    got_net.add_edge(n, docs[k][1])
                
got_net.show("dict.html")

In [27]:
from pyvis.network import Network
import pandas as pd

got_net = Network(height="750px", width="100%", bgcolor="#222222", font_color="white")
got_net.barnes_hut()

#Top10에 대한 노드를 추가
for row in top_user_n.iterrows() :
    src = row[1].title
    got_net.add_node(src, src)
    a = 'movie '+row[1].movieId
    #Actor를 노드 추가
    for i in g[a].keys():
        n = act[act['id'].isin([i[6:]])]['name'].values[0]
        got_net.add_node(n, n)
        got_net.add_edge(src, n)
        #Actor가 출연한 영화에 대한 노드를 만들어서 추가
        for j in g[i].keys():
            for k in range(len(docs)):
                if docs[k][0] == j[6:]:
                    got_net.add_node(docs[k][1], docs[k][1])
                    got_net.add_edge(n, docs[k][1])
                
got_net.show("numpy.html")